# Flights Price Analysis 

The goal of this notebook is to run some analysis on a dataset that contains one-way flights found on Expedia between 2022-04-16 and 2022-10-05 (you can find it at this [link](https://www.kaggle.com/datasets/dilwong/flightprices).

In [1]:
val bucketname = "unibo-bd2223-paolopenazzi"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1676987342002_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2223-paolopenazzi


## DataPreparation

The following columns will be kept and created:
- flightID: Identifier for the flight.
- searchDate: Date the record was obtained from Expedia.
- searchMonth: Month the record was obtained from Expedia.
- searchDay: Day the record was obtained from Expedia.
- flightDate: The date of the flight.
- flightMonth: The month of the flight.
- flightDay: The day of the flight.
- startingAirport: 3-letter code for the starting airport.
- destinationAirport: 3-letter code for the destination airport.
- duration: Travel duration in minutes.
- isEconomy: Is basic economy?
- isRefundable: Is the ticket refundable?
- isNonStop: Is the flight non-stop?
- baseFare: Price of the ticket (not including taxes).
- totalFare: Price of the ticket, including taxes and fees.
- seatsRemaining: Number of remaining seats.
- travelDistance: The total travel distance in miles.

In [2]:
case class FlightData(
    flightID:String,
    searchDate:String,
    searchMonth:String,
    searchDay:String,
    flightDate:String,
    flightMonth:String,
    flightDay:String,
    startingAirport:String,
    destinationAirport:String,
    duration:Int,
    isEconomy:Boolean,
    isRefundable:Boolean,
    isNonStop:Boolean,
    baseFare:Double,
    totalFare:Double,
    seatsRemaining:Int,
    travelDistance:Int
)

object FlightData {

    def parse(line:String) = {
        val input = line.split(",")
        val flightID = input(0)
        val searchDate = input(1)
        val searchMonth = searchDate.substring(5,7)
        val searchDay = searchDate.substring(8,10)
        val flightDate = input(2)
        val flightMonth = flightDate.substring(5,7)
        val flightDay = flightDate.substring(8,10)
        val startingAirport = input(3)
        val destinationAirport = input(4)
        val dur = input(6).replace("P","").replace("T","").split("D|H|M").map(x => x.toInt)
        val duration = dur.length match {
            case 3 => dur(0) * 1440 + dur(1) * 60 + dur(2)
            case 2 => dur(0) * 60 + dur(1)
            case 1 => dur(0)
        }
        val isEconomy = input(8).toBoolean
        val isRefundable = input(9).toBoolean
        val isNonStop = input(10).toBoolean
        val baseFare = input(11).toDouble
        val totalFare = input(12).toDouble
        val seatsRemaining = input(13).toInt
        val travelDistance = input(14) match {
            case "" => 0
            case _ => input(14).toInt
        }
        
        new FlightData(flightID,searchDate,searchMonth,searchDay,flightDate,flightMonth,flightDay,startingAirport,
                       destinationAirport,duration,isEconomy,isRefundable,isNonStop,baseFare,totalFare,
                       seatsRemaining,travelDistance)
    }
}

/** Create RRD */
val rdd = sc.textFile("s3a://"+bucketname+"/datasets/itineraries.csv")
/** Extract header from RDD and parse every row*/
val header = rdd.first(); 
val rddFlights = rdd.filter(row => row != header).map(FlightData.parse)
/** Count elements on RDD */
//rddFlights.count() //82138753

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class FlightData
defined object FlightData
Companions must be defined together; you may wish to use :paste mode for this.
rdd: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2223-paolopenazzi/datasets/itineraries.csv MapPartitionsRDD[1] at textFile at <console>:29
header: String = legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
rddFlights: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[3] at map at <console>:29


#### Visualize data as a DataFrame

In [15]:
import spark.implicits._

val columns = Seq("flightID",
                  "searchDate",
                  "searchMonth",
                  "searchDay",
                  "flightDate",
                  "flightMonth",
                  "flightDay",
                  "startingAirport",
                  "destinationAirport",
                  "duration",
                  "isEconomy",
                  "isRefundable",
                  "isNonStop",
                  "baseFare",
                  "totalFare",
                  "seatsRemaining",
                  "travelDistance")

val flightDataframe = rddFlights.toDF(columns:_*)
flightDataframe.show(60,truncate=40,vertical=true)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
columns: Seq[String] = List(flightID, searchDate, searchMonth, searchDay, flightDate, flightMonth, flightDay, startingAirport, destinationAirport, duration, isEconomy, isRefundable, isNonStop, baseFare, totalFare, seatsRemaining, travelDistance)
flightDataframe: org.apache.spark.sql.DataFrame = [flightID: string, searchDate: string ... 15 more fields]
-RECORD 0----------------------------------------------
 flightID           | 9ca0e81111c683bec1012473feefd28f 
 searchDate         | 2022-04-16                       
 searchMonth        | 04                               
 searchDay          | 16                               
 flightDate         | 2022-04-17                       
 flightMonth        | 04                               
 flightDay          | 17                               
 startingAirport    | ATL                              
 destinationAirport | BOS                              
 duration           | 149                              
 isEc

#### Ignore flights for which there are not at least 7 days of difference between the searchDate and the flightDate

In [3]:
/** import java.time.format.DateTimeFormatter
import java.time.LocalDateTime
import java.time.Duration */
import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate

//type rddType = (String,String,String,String,String,String,String,String,String,Int,Boolean,Boolean,Boolean,Double,Double,Int,Int)

def enoughData(x: FlightData): Long = {
        val searchDate = LocalDate.parse(x.searchDate);
        val flightDate = LocalDate.parse(x.flightDate);
        val daysBetween = DAYS.between(searchDate, flightDate);
        daysBetween
}

val rddFlightsFiltered = rddFlights.filter(x => enoughData(x) >= 7)
// rddFlightsFiltered.count() // 82138753 in RDD, 71823816 in RDD filtered 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate
enoughData: (x: FlightData)Long
rddFlightsFiltered: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[4] at filter at <console>:30


In [4]:
rddFlightsFiltered.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res14: Array[FlightData] = Array(FlightData(15f52084c28c42240e43aec635b05057,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,150,false,false,true,204.0,260.59,0,0), FlightData(ba0813f32f5d4416d272100c19ff0c7e,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,150,false,false,true,338.6,378.6,3,947), FlightData(b3ba48077c67f201e0fbfefbfc18a1cb,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,301,false,false,false,386.05,438.6,2,947), FlightData(84ff1dbb7ffc6b36feb6bf6650be62e1,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,280,false,false,false,395.35,448.6,3,947), FlightData(9ee247d2537e717a034b3542ef2922d2,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,150,false,false,true,431.63,478.6,2,947), FlightData(4e5f81483c4d7a4c2ac56e20c8f1d111,2022-04-16,04,16,2022-04-23,04,23,ATL,BOS,276,false,false,false,486.51,545.1,3,95...

## Data Exploration

In [6]:
val rddFlightsFilteredCached = rddFlightsFiltered.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlightsFilteredCached: rddFlightsFiltered.type = MapPartitionsRDD[4] at filter at <console>:30


In [14]:
//"Number of searches performed: " + rddFlightsFilteredCached.count() // 71823816
//"Number of distinct flights: " + rddFlightsFilteredCached.map(x => x.flightID).distinct().count() // 5475345
//"Number of distinct startingAirport: " + rddFlightsFilteredCached.map(x => (x.startingAirport)).distinct().count() //16
//"Number of flights by starting airport: " + rddFlightsFilteredCached.map(x => (x.flightID, x.startingAirport)).distinct().map(x => (x._2, 1)).reduceByKey(_+_)
// Array((LAX,539111), (CLT,358966), (JFK,264186), (BOS,403703), (OAK,284170), (LGA,392553), (ATL,324273), (MIA,310015), (DTW,300891), (PHL,323567), (SFO,467443), (EWR,257195), (ORD,326516), (DEN,300488), (IAD,277081), (DFW,345298))
//"Number of flights by month: " + rddFlightsFilteredCached.map(x => (x.flightID, x.flightMonth)).distinct().map(x => (x._2, 1)).reduceByKey(_+_)
// Array((04,87897), (05,610532), (06,895364), (07,930405), (08,967938), (09,891707), (10,729489), (11,362013))
// "Number of routes: " + rddFlightsFilteredCached.map(x => (x.startingAirport, x.destinationAirport)).distinct().count()
// numero di voli economi e non economici
//"Number of direct flights: " + rddFlightsFilteredCached.map(x => (x.flightID, x.isNonStop)).distinct().filter(x => x._2 == true).count() // 625655

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddDistinctFlights: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[34] at distinct at <console>:28
res83: String = Number of direct flights: 625655


## Jobs

### 1 - ??

The change that occurs in the ticket price in the 1-7 and 8-14 days prior to flight departure is calculated, dividing the result between 'economy' and non-economy tickets.

The correlation between that change in price and the number of seats remaining on that flight is then displayed.

### 2 - ??

We want to identify the cheapest direct flights departing in a given week, grouped according to the departure airport, calculating the following values:
- average price recorded in the 7 days before the departure of direct flights only
- previous result compared to the distance traveled by the flight, so we find the flights that take us as far as possible for less money

In [4]:
import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate

def daysBetween(x: FlightData): Long = {
    val searchDate = LocalDate.parse(x.searchDate);
    val flightDate = LocalDate.parse(x.flightDate);
    DAYS.between(searchDate, flightDate);
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.time.temporal.ChronoUnit.DAYS
import java.time.LocalDate
daysBetween: (x: FlightData)Long


In [9]:
val rddData1WeekBeforeDirectFlight = rddFlightsFilteredCached.
    filter(x => daysBetween(x) <= 10).
    filter(_.isNonStop).
    cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddData1WeekBeforeDirectFlight: org.apache.spark.rdd.RDD[FlightData] = MapPartitionsRDD[8] at filter at <console>:34


In [10]:
rddData1WeekBeforeDirectFlight.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res17: Long = 1727242


In [14]:
val rddAveragePricePerFlight = rddData1WeekBeforeDirectFlight.
    map(x => (x.flightID, x.baseFare)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerFlight: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[14] at map at <console>:33


In [15]:
val rddAveragePricePerFlightWithDistance = rddData1WeekBeforeDirectFlight.
    map(x => (x.flightID, x.travelDistance)).
    distinct().
    join(rddAveragePricePerFlight)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerFlightWithDistance: org.apache.spark.rdd.RDD[(String, (Int, Double))] = MapPartitionsRDD[21] at join at <console>:35


In [21]:
val rddAveragePricePerDistance = rddAveragePricePerFlightWithDistance.
    map({case(x,y) => (x, y._2/y._1)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerDistance: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[22] at map at <console>:31


In [22]:
val rddAveragePricePerAirport = rddData1WeekBeforeDirectFlight.
    map(x => (x.flightID, x.startingAirport)).
    distinct().
    join(rddAveragePricePerDistance)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddAveragePricePerAirport: org.apache.spark.rdd.RDD[(String, (String, Double))] = MapPartitionsRDD[29] at join at <console>:35


In [23]:
val rddFinal = rddAveragePricePerAirport.
    map({case(x,y) => (y._1, y._2)}).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (v._1/v._2, k)}).
    sortByKey().
    collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFinal: Array[(Double, String)] = Array((Infinity,LAX), (Infinity,JFK), (Infinity,LGA), (Infinity,MIA), (Infinity,PHL), (Infinity,EWR), (Infinity,ORD), (Infinity,IAD), (Infinity,CLT), (Infinity,BOS), (Infinity,OAK), (Infinity,ATL), (Infinity,DTW), (Infinity,SFO), (Infinity,DEN), (Infinity,DFW))


In [18]:
val rddAveragePricePerAirport = rddData1WeekBeforeDirectFlight.
    map(x => (x.startingAirport, x.baseFare)).
    aggregateByKey((0.0,0.0))((x,y)=>(x._1+y, x._2 + 1) , (x,y)=>(x._1 + y._1, x._2 + y._2)).
    map({case(k,v) => (k, v._1/v._2)}).
    collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res28: Array[(String, Double)] = Array((LAX,323.65968300851813), (CLT,271.61687347318605), (JFK,249.8101657160011), (BOS,203.0790532989199), (OAK,114.69472489082972), (LGA,173.16174405059084), (ATL,250.04267252195743), (MIA,197.6175176848874), (DTW,255.66425100821132), (PHL,237.64767706302794), (SFO,343.6562917529396), (EWR,226.05609893387003), (ORD,211.3787435584654), (DEN,239.2157848786453), (IAD,262.81525903041825), (DFW,238.9464623652575))
